In [ ]:
# notebook settings
%load_ext autoreload
%autoreload 2
# %matplotlib notebook

# external imports
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt

# internal imports
from pympc.control.hybrid_benchmark.mixed_logical_dynamical_system import MixedLogicalDynamicalSystem
from pympc.control.hybrid_benchmark.controller_mld import HybridModelPredictiveController
from pympc.plot import plot_input_sequence, plot_state_trajectory, plot_output_trajectory

In [ ]:
# system numeric parameters
mc = 1.   # mass cart
mp = 1.   # mass pole
l  = 1.   # length pole
d  = .5   # distance walls from origin
k  = 100. # stiffness contacts
nu = 30.  # damping contacts
g  = 10.  # gravity acceleration
h  = .05  # integration time step

# state bounds
x_max = np.array([d, np.pi/8., 2., 1.])
x_min = - x_max

# input bounds
uc_max = np.array([2.])
uc_min = - uc_max

In [ ]:
# symbolic states
x = sp.Matrix(sp.symbols('q t qd td'))

# symbolic continuous inputs
uc = sp.Matrix([sp.symbols('u')])

# symbolic binary inputs
ub = sp.zeros(0,1)

# symbolic forces (left, right)
sc = sp.Matrix(sp.symbols('f_l f_r'))

# symbolic auxiliary binaries (left, right)
sb = sp.Matrix(sp.symbols('el_l dam_l el_r dam_r'))

# dynamics
x2_dot = x[1]*g*mp/mc + uc[0]/mc
x3_dot = x[1]*g*(mc+mp)/(l*mc) + uc[0]/(l*mc) + (sc[1] - sc[0])/(l*mp)
x_next = sp.Matrix([
    x[0] + h*x[2],
    x[1] + h*x[3],
    x[2] + h*x2_dot,
    x[3] + h*x3_dot
])

In [ ]:
# enforce state constraints
state_upper_bound = x - x_max.reshape(x_max.size, 1)
state_lower_bound = x_min.reshape(x_min.size, 1) - x

# enforce input constraints
input_upper_bound = uc - uc_max.reshape(uc_max.size, 1)
input_lower_bound = uc_min.reshape(uc_min.size, 1) - uc

In [ ]:
f = { # force
    'l': sc[0],
    'r': sc[1]
}
el = { # inidicator for penetation
    'l': sb[0],
    'r': sb[2]
}
dam = { # inidicator for pulling
    'l': sb[1],
    'r': sb[3]
}
p = { # penetration
    'l': - d - x[0] + l*x[1],
    'r': x[0] - l*x[1] - d
}
p_dot = { # relative velocity
    'l': - x[2] + l*x[3],
    'r': x[2] - l*x[3]
}
p_min = { # min penetration
    'l': - d - x_max[0] + l*x_min[1],
    'r': x_min[0] - l*x_max[1] - d
}
p_max = { # max penetration
    'l': - d - x_min[0] + l*x_max[1],
    'r': x_max[0] - l*x_min[1] - d
}
p_dot_min = { # min relative velocity
    'l': - x_max[2] + l*x_min[3],
    'r': x_min[2] - l*x_max[3]
}
p_dot_max = { # max relative velocity
    'l': - x_min[2] + l*x_max[3],
    'r': x_max[2] - l*x_min[3]
}
f_min = { # min force
    'l': k*p_min['l'] + nu*p_dot_min['l'],
    'r': k*p_min['r'] + nu*p_dot_min['r']
}
f_max = { # max force
    'l': k*p_max['l'] + nu*p_dot_max['l'],
    'r': k*p_max['r'] + nu*p_dot_max['r']
}

In [ ]:
# contact constraints big-M method
contacts = []
for lr in ['l', 'r']:
    
    # el = 1 if p >= 0, el = 0 if otherwise
    contacts.append(sp.Matrix([p_min[lr] * (1. - el[lr]) - p[lr]]))
    contacts.append(sp.Matrix([p[lr] - p_max[lr] * el[lr]]))
    
    # dam = 1 if k p + nu p_dot >= 0, dam = 0 if otherwise
    contacts.append(sp.Matrix([f_min[lr] * (1. - dam[lr]) - k*p[lr] - nu*p_dot[lr]]))
    contacts.append(sp.Matrix([k*p[lr] + nu*p_dot[lr] - f_max[lr] * dam[lr]]))
    
    # el = 0 or dam = 0 implies f = 0
    contacts.append(sp.Matrix([- f[lr]]))
    contacts.append(sp.Matrix([f[lr] - f_max[lr]*el[lr]]))
    contacts.append(sp.Matrix([f[lr] - f_max[lr]*dam[lr]]))
    
    # el = dam = 1 implies f = k p + nu p_dot
    contacts.append(sp.Matrix([k*p[lr] + nu*p_dot[lr] + nu*p_dot_max[lr]*(el[lr]-1.) - f[lr]]))
    contacts.append(sp.Matrix([f[lr] - k*p[lr] - nu*p_dot[lr] - f_min[lr]*(dam[lr]-1.)]))

In [ ]:
# gather constraints
constraints = sp.Matrix([
    state_upper_bound,
    state_lower_bound,
    input_upper_bound,
    input_lower_bound,
    sp.Matrix(contacts)
])

In [ ]:
# construct MLD system
variables = {'x':x, 'uc':uc, 'ub':ub, 'sc':sc, 'sb':sb}
MLD = MixedLogicalDynamicalSystem.from_symbolic(variables, x_next, constraints)

In [ ]:
# controller parameters
N = 30
weight_matrices = {}
weight_matrices['Q'] = 10.*np.eye(MLD.nx)*h
weight_matrices['R'] = 1.*np.eye(MLD.nuc)*h
weight_matrices['P'] = weight_matrices['Q']#*100.

# hybrid controller
controller = HybridModelPredictiveController(MLD, N, weight_matrices)

In [ ]:
x0 = np.array([0., 0., 1., 0.])
# x0 = np.array([0.44366632, 0.06866632, 2.94079905, 0.45738834])

# try controller open loop

In [ ]:
cost, sol, n_nodes, leaves = controller.feedforward(x0)#, tree_file_name='tau_0')

In [ ]:
stage_variables = {
    'x_0' : sol['primal']['x'][0],
    'uc_0' : sol['primal']['uc'][0],
    'ub_0' : sol['primal']['ub'][0],
    'sc_0' : sol['primal']['sc'][0],
    'sb_0' : sol['primal']['sb'][0],
    'e_0' : np.zeros(MLD.nx)
}
warm_start = controller.construct_warm_start(leaves, stage_variables)
cost_1_ws, sol_1_ws, n_nodes_1_ws, leaves_1_ws = controller.feedforward(
    sol['primal']['x'][1],
#     tree_file_name='tau_1_e_0',
    warm_start=warm_start
)

In [ ]:
np.random.seed(1)
e_norms = [0.003, 0.01, 0.03, 0.1]
e_0 = np.random.randn(MLD.nx)
for e_norm in e_norms:
    e_0 *= e_norm / np.linalg.norm(e_0)
    print e_0, np.linalg.norm(e_0)
    stage_variables = {
        'x_0' : sol['primal']['x'][0],
        'uc_0' : sol['primal']['uc'][0],
        'ub_0' : sol['primal']['ub'][0],
        'sc_0' : sol['primal']['sc'][0],
        'sb_0' : sol['primal']['sb'][0],
        'e_0' : e_0
    }
    warm_start = controller.construct_warm_start(leaves, stage_variables)
    cost_1_ws, sol_1_ws, n_nodes_1_ws, leaves_1_ws = controller.feedforward(
        sol['primal']['x'][1] + e_0,
#         tree_file_name='tau_1_e_%.3f'%e_norm,
        warm_start=warm_start
    )

In [ ]:
np.random.seed(1)
e_norms = [0.003, 0.01, 0.03, 0.1]
e_0 = np.random.randn(MLD.nx)
for e_norm in e_norms:
    e_0 *= e_norm / np.linalg.norm(e_0)
    print e_0, np.linalg.norm(e_0)
    cost_1, sol_1, n_nodes_1, leaves_1 = controller.feedforward(
        sol['primal']['x'][1] + e_0,
#         tree_file_name='tau_1_e_0'
    )

In [ ]:
plot_input_sequence(sol['primal']['uc'], h, (uc_min, uc_max))
# plt.savefig('input.pdf')

In [ ]:
plot_state_trajectory(sol['primal']['x'], h, (x_min, x_max))
# plt.savefig('state.pdf')

In [ ]:
plot_input_sequence(sol['primal']['sc'], h)
# plt.savefig('forces.pdf')

In [ ]:
plot_input_sequence(sol['primal']['sb'], h)
# plt.savefig('binaries.pdf')

In [ ]:
# output (horizontal position of the tip of the pole)
C = np.array([[1., -l, 0., 0.]])
y_min = np.array([-d])
y_max = np.array([d])
plot_output_trajectory(C, sol['primal']['x'], h, (y_min, y_max))
plt.savefig('output.pdf')

# solve in closed loop

In [ ]:
np.random.seed(1)
N_sim = 50
u_sim = []
x_sim = [x0]
warm_start = None
for t in range(N_sim):
    print('Time step ' + str(t) + '.\r'),
    cost, sol, n_nodes, leaves = controller.feedforward(
        x_sim[-1],
        warm_start=warm_start,
        printing_period=None
    )
    u_sim.append(sol['primal']['uc'][0])
    x_sim.append(sol['primal']['x'][1])
#     print cost
    print len(leaves)
    
    stage_variables = {
        'x_0' : sol['primal']['x'][0],
        'uc_0' : sol['primal']['uc'][0],
        'ub_0' : sol['primal']['ub'][0],
        'sc_0' : sol['primal']['sc'][0],
        'sb_0' : sol['primal']['sb'][0],
        'e_0' : np.zeros(MLD.nx)
    }
    warm_start = controller.construct_warm_start(leaves, stage_variables)

In [ ]:
# plot_input_sequence(e_sim, h, (-e_max*np.ones(MLD.nx), e_max*np.ones(MLD.nx)))
# plt.show()
plot_input_sequence(u_sim, h, (uc_min, uc_max))
plt.show()
plot_state_trajectory(x_sim, h, (x_min, x_max))
plt.show()

In [ ]:
C = np.array([[1., -l, 0., 0.]])
y_min = np.array([-d])
y_max = np.array([d])
plot_output_trajectory(C, x_sim, h, (y_min, y_max))
plt.savefig('output.pdf')

# fancy plot

In [ ]:
np.random.seed(1)
# e_norms = [0., 0.005, 0.01, 0.02]
e_norms = [0.]#, 0.003, 0.01, 0.03, 0.1]
N_sim = 50

nodes = {}
nodes_ws = {}
len_ws = {}
errors = {}


for e_norm in e_norms:
    
    nodes[e_norm] = []
    nodes_ws[e_norm] = []
    len_ws[e_norm] = []
    errors[e_norm] = []
    
    x_sim = [x0]
    ws = None
    
    for t in range(N_sim):

        print 'Time step %d\n' % t
        
        # solve without warm start
        print 'Solution without warm start:\n'
        cost, sol, n_nodes, leaves = controller.feedforward(
            x_sim[-1],
#             printing_period=None
        )
        nodes[e_norm].append(n_nodes)
        
        # solve with warm start
        print 'Solution with warm start:\n'
        cost_ws, sol_ws, n_nodes_ws, leaves_ws = controller.feedforward(
            x_sim[-1],
            warm_start=ws,
#             printing_period=None
        )
        nodes_ws[e_norm].append(n_nodes_ws)
        
        # check results
        if cost is None:
            break
        assert np.isclose(cost, cost_ws)
        
        # generate random error of specified norm
        e_t = np.random.randn(MLD.nx)
        e_t *= e_norm / np.linalg.norm(e_t)
        errors[e_norm].append(e_t)
        print e_t, np.linalg.norm(e_t)
        print '\n'
        
        #generate warm start
        stage_variables = {
            'x_0' : sol['primal']['x'][0],
            'uc_0' : sol['primal']['uc'][0],
            'ub_0' : sol['primal']['ub'][0],
            'sc_0' : sol['primal']['sc'][0],
            'sb_0' : sol['primal']['sb'][0],
            'e_0' : e_t
        }
        ws = controller.construct_warm_start(leaves_ws, stage_variables)
        len_ws[e_norm].append(len(ws))
        
        # update state
        x_sim.append(sol['primal']['x'][1] + e_t)

In [ ]:
np.random.seed(1)

# pick colors
colors = {
    e_norms[0]: '#1f77b4',
    e_norms[1]: '#ff7f0e',
    e_norms[2]: '#2ca02c',
    e_norms[3]: '#d62728',
    e_norms[4]: '#9467bd'
}

plt.figure(figsize=(9,5))

# plot trends
for e_norm in e_norms:
    plt.scatter(range(len(nodes[e_norm])), nodes[e_norm], color=colors[e_norm], marker='+')
    plt.scatter(range(1,len(nodes_ws[e_norm])), nodes_ws[e_norm][1:], color=colors[e_norm], marker='x')
    plt.scatter(range(len(len_ws[e_norm])), len_ws[e_norm], color=colors[e_norm], marker='.')
plt.gca().set_ylim(bottom=-1)

# create legend
plt.scatter(N_sim+2, 0, marker='+', color='k', label='Cold started')
plt.scatter(N_sim+2, 0, marker='x', color='k', label='Warm started')
plt.scatter(N_sim+2, 0, marker='.', color='k', label='Warm-start size')
for e_norm in e_norms:
    plt.scatter(N_sim+2, 0, color=colors[e_norm], label=r'$||e_t||=%.3f$'%e_norm)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# others
plt.xlabel(r'Time step $t$')
plt.ylabel('Number of subproblems')
plt.xlim(-1, N_sim+1)
plt.yscale('symlog')
plt.yticks([i*j for i in range(11) for j in [1, 10, 100]])
plt.ylim(5, 1e3)
plt.grid(True)
plt.tight_layout()
plt.gca().set_axisbelow(True)
# plt.savefig('trends.pdf')

np.save('nodes', nodes)
np.save('nodes_ws', nodes_ws)
np.save('len_ws', len_ws)
np.save('errors', errors)

In [ ]:
# plot trends
for e_norm in e_norms:
    
#     for i in range(N_samples):
    
#     plt.figure(figsize=(8,4))

    plt.plot(
        range(len(nodes[e_norm])),
        nodes[e_norm],
        color='#1f77b4',
#         marker='+',
        label='Cold started'
    )
    plt.plot(
        range(1,len(nodes_ws[e_norm])),
        nodes_ws[e_norm][1:],
        color='#ff7f0e',
#         marker='x',
        label='Warm started'
    )
    plt.plot(
        range(len(len_ws[e_norm])),
        len_ws[e_norm],
        color='#2ca02c',
#         marker='.',
        label='Warm-start size'
    )


    # others
    plt.xlabel(r'Time step $t$')
    plt.ylabel('Number of subproblems')
    plt.xlim(0, N_sim)
    plt.yscale('symlog')
    plt.yticks([i*j for i in range(11) for j in [1, 10, 100]])
    plt.ylim(5, 1e3)
    plt.grid(True)
    plt.legend(loc=9)
    plt.tight_layout()
    plt.gca().set_axisbelow(True)
    plt.title(r'$|| e_t || = %.3f$'%e_norm)
    plt.show()
#     plt.savefig('trends_%.3f.pdf'%e_norm)
# plt.savefig('trends_nominal.pdf')

# statistical analysis

In [ ]:
e_norms = [0.003, 0.01, 0.03, 0.1]
N_sim = 50
N_samples = 100

nodes = {}
nodes_ws = {}
len_ws = {}
errors = {}

for e_norm in e_norms:
    
    print 'Error norm %d\n'%e_norm
    
    nodes[e_norm] = []
    nodes_ws[e_norm] = []
    len_ws[e_norm] = []
    errors[e_norm] = []
    i = 0
    
    while len(nodes[e_norm]) < N_samples:
        np.random.seed(i)
        
        print '\nRollout %d\n'%len(nodes[e_norm])
        rollout_success = True
        
        nodes_i = []
        nodes_ws_i = []
        len_ws_i = []
        errors_i = []
    
        x_sim = [x0]
        ws = None

        for t in range(N_sim):
            print 'Time step %d'%t, 

            # solve without warm start
            try:
                cost, sol, n_nodes, leaves = controller.feedforward(
                    x_sim[-1],
                    printing_period=None
                )
            except:
                rollout_success = False
                print '\nUnseccessful rollout'
                break
            nodes_i.append(n_nodes)
            print n_nodes, 

            # solve with warm start
            try:
                cost_ws, sol_ws, n_nodes_ws, leaves_ws = controller.feedforward(
                    x_sim[-1],
                    warm_start=ws,
                    printing_period=None
                )
            except:
                rollout_success = False
                print '\nUnseccessful rollout'
                break
            nodes_ws_i.append(n_nodes_ws)
            print n_nodes_ws,

            # check results
            if cost is None:
                break
            assert np.isclose(cost, cost_ws)

            # generate random error of specified norm
            e_t = np.random.randn(MLD.nx)
            e_t = np.multiply(e_t, x_max)
            e_t *= e_norm / np.linalg.norm(e_t)
            errors_i.append(e_t)

            #generate warm start
            stage_variables = {
                'x_0' : sol['primal']['x'][0],
                'uc_0' : sol['primal']['uc'][0],
                'ub_0' : sol['primal']['ub'][0],
                'sc_0' : sol['primal']['sc'][0],
                'sb_0' : sol['primal']['sb'][0],
                'e_0' : e_t
            }
            ws = controller.construct_warm_start(leaves_ws, stage_variables)
            len_ws_i.append(len(ws))
            print len(ws)

            # update state
            x_sim.append(sol['primal']['x'][1] + e_t)
            
        if rollout_success:
            nodes[e_norm].append(nodes_i)
            nodes_ws[e_norm].append(nodes_ws_i)
            len_ws[e_norm].append(len_ws_i)
            errors[e_norm].append(errors_i)
            np.save('nodes_%.3f'%e_norm, nodes[e_norm])
            np.save('nodes_ws_%.3f'%e_norm, nodes_ws[e_norm])
            np.save('len_ws_%.3f'%e_norm, len_ws[e_norm])
            np.save('errors_%.3f'%e_norm, errors[e_norm])
        else:
            controller = HybridModelPredictiveController(MLD, N, weight_matrices)
            
        i += 1
        